엑셀에 코드저장을 위해서 만들었음

In [13]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.kmeans import MyKmeans
from sklearn.cluster import KMeans

In [42]:
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\Use_data\\재무\\")

kind_data = tools.dict_data_preprocess(kind_data, function=np.sum, window_size='M',fillna=True)
financial_data = tools.dict_data_preprocess(financial_data, function=np.mean, window_size='M',fillna=False)

In [43]:
concat_data = tools.dict_data_concat(kind_data,financial_data, dropna=False)
plus_data = tools.dict_data_plus(concat_data, plus_one="BW금액_전체시장",plus_two="CB금액_전체시장", return_origianl=False)
plus_data2 = tools.dict_data_plus(plus_data, plus_one="상호변경카운트_전체시장",plus_two="업종변경카운트_전체시장", return_origianl=False)
plus_data3 = tools.dict_data_plus(plus_data2, plus_one="타법인주식및출자증권취득결정금액_전체시장",plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
plus_data4 = tools.dict_data_plus(plus_data3, plus_one="투자환기종목지정카운트_전체시장",plus_two="투자주의카운트_전체시장", return_origianl=False)
plus_data5 = tools.dict_data_plus(plus_data4, plus_one="조회공시요구카운트_전체시장",plus_two="최대주주변경카운트_전체시장", return_origianl=False)
divide_data = tools.dict_data_divide(plus_data5,divide_what=["타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장",
                                                             "BW금액_전체시장+CB금액_전체시장", "자기주식처분결정금액_전체시장","당기순이익",
                                                             "매출채권",'매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'],
                                                               divide_into="총자본", return_origianl=False)
scaled_data = tools.dict_data_scale(divide_data, scaler=scaler.minmax)
drop_data = tools.dict_data_drop(scaled_data, dropna=True)     

c:\Jehyeon\2022_Winter_KRX\KRX\krx_fr\preprocess\scaler.py:37: RuntimeWarning: All-NaN slice encountered
  min = np.nanmin(arr)
c:\Jehyeon\2022_Winter_KRX\KRX\krx_fr\preprocess\scaler.py:38: RuntimeWarning: All-NaN slice encountered
  max = np.nanmax(arr)
c:\Jehyeon\2022_Winter_KRX\KRX\krx_fr\preprocess\scaler.py:39: RuntimeWarning: invalid value encountered in true_divide
  return (arr-min) / (max - min)


In [48]:
concat_data["2012-01-01"].columns

Index(['BW금액_전체시장', 'CB금액_전체시장', '불성실공시법인지정카운트_전체시장', '상호변경카운트_전체시장',
       '업종변경카운트_전체시장', '자기주식처분결정금액_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정금액_전체시장', '투자주의카운트_전체시장', '투자환기종목지정카운트_전체시장',
       '특수관계인에대한자금대여금액_전체시장', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [44]:
drop_data["2012-01-01"].columns

Index(['타법인주식및출자증권취득결정금액_전체시장+특수관계인에대한자금대여금액_전체시장/총자본',
       'BW금액_전체시장+CB금액_전체시장/총자본', '자기주식처분결정금액_전체시장/총자본', '당기순이익/총자본',
       '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본', '판매 및 관리비/총자본',
       '조회공시요구카운트_전체시장+최대주주변경카운트_전체시장', '투자환기종목지정카운트_전체시장+투자주의카운트_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', '불성실공시법인지정카운트_전체시장', '차입금의존도', '최대주주비율'],
      dtype='object')

In [47]:
# 클러스터링 할 sample data 하나 생성
sample_2022 = drop_data["2022-01-01"]
company = drop_data["2022-01-01"].index.to_list()

kmeans = KMeans(n_clusters=5, n_init=100, max_iter=10000)
kmeans.fit(sample_2022)

cluster_df = pd.DataFrame(data=np.c_[company,kmeans.labels_], columns=["기업공시코드","클러스터"])
cluster_df["클러스터"].value_counts()

1    785
4    612
2    579
3    331
0     39
Name: 클러스터, dtype: int64